### Local alignment using the Smith-Waterman algorithm ###


To succesfully programme the Smith-Waterman algorithm three main functions have been necessary to create. In the following I will explain the role of each and the principle of how they are scripted

#### Function 1: Handling the fasta file and dividing each sequence in a dictionary
This defined function is named <font color='blue'>create_seq_dict( )</font>. It's a function that opens the given fasta file and generates a dictionary with each sequence of the file. <br>
**Line 2-4**: The function firstly makes a dictionary named _d_. A dictionary contains keys and values, which here are named _header_ and _sequence_ respectively. _header_ and _sequence_ are, to begin with, both empty strings. <br> 

**Line 5-8**: The function opens the given fasta file, with the functions called <font color='green'>with</font> and <font color='green'>open( )</font>, which allows the programme to acces the content of the file. Here after a forloop loops over each line. All letters are converted into upper case letters with <font color='green'>.upper( )</font>, and the new line command (/n) of the end of each line is stripped of with <font color='green'>.rstrip( )</font>. <br>

**Line 9-16**: The function uses an if/else statement to define if a line is a header of a sequence. If it is then the line is assigned to variable _header_ (line 13), and the _sequence_ string is empty (line 14). Else it will add each line to the _sequence_ string (line 15-16). In line 10, there is again an if statement. If _header_ is not empty, next time the line starts with ">" (at the next sequence), then the _sequence_ is equal to key in dictionary, that is the current _header_. This means that the first header is the key to the following sequence. By adding the if statement in line 10, the first sequence of the fasta file will not be skipped. <br>

**Line 18-19**: When all lines have been looped over, the last _sequence_ is equal to key in dictionary, that is the last _header_. The dictionary is returned, which stops the function from running more, and makes it possible to use the dictionary in another function.<br>


In [1]:
import numpy as np
import pandas as pd

In [2]:
path = "/Users/test_seq.fasta" #path for input fasta file
path_ = "/Users/alignment_" #path for alignment results

In [3]:
def create_seq_dict(file_name):
    d = {}
    header = ""
    sequence = ""
    with open(file_name) as f:
        for line in f:
            line = line.upper()
            line = line.rstrip()
            if line[0] == ">":
                if header != "":
                    d[header] = sequence

                header = line
                sequence = ""
            else:
                sequence += line

    d[header] = sequence
    return d

#### Function 2: Creating the scoring matrix
This defined function is named <font color='blue'>scoring( )</font>. It is a function that first initiallizes a scoring matrix based on the length of input sequences. The initialized scoring matrix consists of only zeros, but the function will go through all positions and fill them all in according to the algorithm.<br>
**Line 2-4**: From <font color='blue'>create_seq_dict( )</font> a dictionary named _seq_ is made. This is used to define the two sequences by calling the two keys (*seq1_name*, *seq2_name*). These two are named *seq1_* and *seq2_* and they are strings.<br>

**Line 6-12**: To make it more easy to work with later on, the two strings are converted into lists by looping over each position and appending them to a new variable (called *seq1* / *seq2*). The two lists are generated so they start with a zero, which is important for the algorithm.  <br>

**Line 14-15**: A scoring matrix of zeros is made using the function np.zeros( ) (SciPy 2019a), which is part of the numpy (np) package installed at the very top of the script. It uses the lengths of _seq1_ and _seq2_. In line 15, the dimesions of the scoring matrix are defined using the .shape() function for np arrays (SciPy 2019b). <br>

**Line 16-17**: By using the two dimensions of the scoring matrix, a forloop is created to go in the i and j direction. i follows _seq1_ and j follows _seq2_. They both go from posisiton 1 (skipping position 0) to the end of the sequence. Now i and j can be used as coordinates, that will change as the forloop goes through the matrix. <br>

**Line 18-20**: Three variables are defined, so we have the above, the left and the above left value for a given position by using the i and j values. <br>

**Line 21-22**: The function will break if i or j reach their limit by hitting the end of the sequence. <br>

**Line 24-31**: An if/else statement, that if the i/j position of _seq1_ and _seq2_ are equal, then adds the match score (ms) to the above left value, and a gap penality (w) score to the above and left value. Else, the statement adds the mismatch score (mms) to the above left value, and a gap penality score to the above and left value.  <br>

**Line 33-34**: <font color='green'>max( )</font> takes the maximum value of the four options, left postition, above position, above left position and zero like equation (1). The current position of the forloop (starting in line 16), gets the maximum value assigned. <br>

**Line 36**: The scoring matrix is returned, which stops the function from running more, and makes it possible to use the scoring matrix in another function. 

In [4]:
def scoring(file_name, seq1_name, seq2_name, w, ms, mms):
    seq = create_seq_dict(file_name)
    seq1_ = seq[seq1_name]
    seq2_ = seq[seq2_name]
    
    seq1 = [0,]
    for pos in seq1_:
        seq1.append(pos)

    seq2 = [0,]
    for pos in seq2_:
        seq2.append(pos)
    
    scoring_matrix = np.zeros((len(seq1),len(seq2)))
    dim = scoring_matrix.shape
    for i in range(1, dim[0]):
        for j in range(1, dim[1]):
            above_left = scoring_matrix[i-1][j-1]
            left = scoring_matrix[i][j-1]
            above = scoring_matrix[i-1][j]
           
            if i >= len(seq1) or j >= len(seq2):
                break
            if seq1[i] == seq2[j]:
                above_left += ms
                left -= w
                above -= w
            else: 
                above_left += mms 
                left -= w
                above -= w
            
            max_value = max(left, above, above_left, 0)
            scoring_matrix[i][j] = max_value
            
    return scoring_matrix

#### Function 3: Aligning the sequences
This defined function is named <font color='blue'>align( )</font>. It is a function that takes the scoring matrix, finds the highest possible score, and backtracks in the matrix  so the two sequences can be aligned. 

**Line 2:** From <font color='blue'>scoring( )</font> a scoring matrix is defined. Here it is called *scoring_matrix_result*.  <br>

**Line 4-14:** Repetition of line 2-12 in function 2.  <br>

**Line 16-20**: The maximum of the scoring matrix is found with the numpy function .amax() (thispointer.com 2018a). Then the position of the maximum is found with the numpy function .where() (thispointer.com 2018b). The results of the .where() function are converted into a list for easier use, where each coordinate is in one bracket. This list is called *max_where*.
Line 17 and 20 will print text as part of the results, when running the code. <br>

**Line 22-26:** Two empty alignment lists (*alignment1* and *alignment2*) are made, to be filled by the function. *alignment1* for _seq1_ and _alignment2_ for _seq2_. Then there is a third list called *alignment1_2* which will be used two combine _alignment1_ and _alignment2_. Further more, there is _alignment_ which is a dictionary which is necessary because one scoring matrix can have multiple alignments. _key_ is the counter of alignments, so that each alignment of a scoring matrix can be called separatly. <br>

**Line 28 - 30**: A forloop that goes through each coordinate in the *max_where*. i and j are assigned the values corresponding the coordinate. <br>

**Line 32-54:** This is the whileloop that bracktracks through the scoring matrix and assign the right letters to the alignment. It runs as long as the position has a value over 0. 
To make sure that the backtracking is done in the same route as the scoring, one if and three elif statements are made. As example, the first two statements are explained: <br>
if the left above value + match score = value for current position, then there is a match. The i and j position of both sequences are appended to each their own list. The i and j values both become one smaller, so that the loop now starts in the position left above.<br>
Else, if the above position - gap penalty = value for current position, then it means there has to be a gap in the alignment. The i position in _seq1_ is appended to _alignment1_ , while _alignment2_ gets a gap (“-”). i gets reduced by one, while j stays the same, so the loop now starts in the position above.<br> 

**Line 56-57:** The two alignments are combined into *alignment1_2*. *alignment1_2* is put into a dictionary with 0 being the key. <br>

**Line 58- 60:** The key is added up by one, and the two _alignment1_ and _alignment2_ lists are reset to be empty lists. Then the loop can take the next coordinate in *max_where* (if any). <br>

**Line 62-66:** To be able to visualise the alignment(s) for the scoring matrix, the result are saved to an excel file using the Panda function .to_excel( ) (Willems 2016). Panda (pd) is a package which is installed in the very top of the script. A forloop is made, so if there are more than one alignment, each file will get their own name (ending with different numbers).

In [5]:
def align(file_name, seq1_name, seq2_name, w, ms, mms):
    scoring_matrix_result = scoring(file_name, seq1_name, seq2_name, w, ms, mms)
    
    seq = create_seq_dict(file_name)
    seq1_ = seq[seq1_name]
    seq2_ = seq[seq2_name]
    
    seq1 = [0,]
    for pos in seq1_:
        seq1.append(pos)

    seq2 = [0,]
    for pos in seq2_:
        seq2.append(pos)
        
    maximum = np.amax(scoring_matrix_result, axis=None)
    print("Score of alignment:", maximum)
    max_where = np.where(scoring_matrix_result == maximum)
    max_coordinate = list(zip(max_where[0], max_where[1]))
    print(len(max_coordinate), "possible alignment(s) with this score, with chosen parameters. Results stored in chosen pathway")

    alignment1 = []
    alignment2 = []
    alignment1_2 = []
    alignment = {}
    key = 0
    
    for pos in range(len(max_coordinate)):
        i = max_coordinate[pos][0]
        j = max_coordinate[pos][1]
        
        while scoring_matrix_result[i][j] > 0: 
            if scoring_matrix_result[i-1][j-1] + ms == scoring_matrix_result[i][j]:
                alignment1.append(seq1[i])
                alignment2.append(seq2[j]) 
                i -= 1
                j -= 1
                                    
            elif scoring_matrix_result[i-1][j] - w == scoring_matrix_result[i][j]:
                alignment1.append(seq1[i])
                alignment2.append("-")
                i -= 1
                
    
            elif scoring_matrix_result[i][j-1] - w == scoring_matrix_result[i][j]:
                alignment1.append("-")
                alignment2.append(seq2[j])
                j -= 1      
                
            elif scoring_matrix_result[i-1][j-1] + mms == scoring_matrix_result[i][j]:
                alignment1.append(seq1[i])
                alignment2.append(seq2[j])
                i -= 1
                j -= 1
                
        alignment1_2 = [alignment1, alignment2]
        alignment[key] = alignment1_2
        key += 1
        alignment1 = []
        alignment2 = []
    
    for x in range(len(max_coordinate)):
        alignment_results = pd.DataFrame(alignment[x]) 
        number = str(x)
        result_path = path_ + number + ".xlsx"
        alignment_results.to_excel(result_path)

### Alignment

The following code is used to give the resulting alignment(s). The user can change the parameters of the function, with a match score, mismatch score, and gap penalty (linear) of own choice. The fasta file can be changed in the top of the script ("path"), and sequence names can also easily be changed, as long as the squence names are in upper case letters. 

In [6]:
align(path,'>TEST_SEQ_1', '>TEST_SEQ2', 2, 8, -5)

Score of alignment: 1580.0
1 possible alignment(s) with this score, with chosen parameters. Results stored in chosen pathway


The resulting alignment(s) are stored in an excel file in a path of the user's choice. This path can be edited in the top of the script ("path_"). The resulting excel file contains two rows, one with the letters of sequence 1 and one with letters of sequence 2, aligning correctly. If there is a gap, it will be marked with "-".